In [18]:
!pip install langchain-openai langchain-chroma
!pip install -U langchain langchain-community
!pip install python-dotenv

In [19]:
%%writefile .env
MY_SECRET_KEY = 12345

Overwriting .env


In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [22]:
#!pip uninstall -y pandas
#!pip install --no-cache-dir pandas==2.2.2


In [23]:
!pip check

ipython 7.34.0 requires jedi, which is not installed.


In [24]:
import pandas as pd
books = pd.read_csv("/content/books_cleaned (1).csv")

In [25]:
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883A NOVEL THAT READERS and critics ...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982A new 'Christie for Christmas' --...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736A memorable, mesmerizing heroine ..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897Lewis' work on the nature of love...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934""In The Problem of Pain, C.S. Lew..."


In [26]:
books["tagged_description"]

,tagged_description
0,9780002005883A NOVEL THAT READERS and critics ...
1,9780002261982A new 'Christie for Christmas' --...
2,"9780006178736A memorable, mesmerizing heroine ..."
3,9780006280897Lewis' work on the nature of love...
4,"9780006280934""In The Problem of Pain, C.S. Lew..."
...,...
5192,9788172235222On A Train Journey Home To North ...
5193,9788173031014This book tells the tale of a man...
5194,9788179921623Wisdom to Create a Life of Passio...
5195,9788185300535This collection of the timeless t...


In [27]:
books["tagged_description"].to_csv("tagged_description.txt", sep="\n", index=False, header=False)

In [28]:
raw_documents = TextLoader("/content/tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Streaming output truncated to the last 5000 lines.


In [29]:
documents[0]

Document(metadata={'source': '/content/tagged_description.txt'}, page_content='9780002005883A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s det

In [30]:
!pip install chromadb sentence-transformers


In [31]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Use a free open-source embedding model
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Load documents into Chroma
db_books = Chroma.from_documents(documents, embedding=embedding)


In [ ]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k=10)
docs

## the problem here is that it is giving recomendation on description but we want recomendation on book's title and authors

In [32]:
isbn_candidate = docs[0].page_content.split()[0].strip()

isbn_numeric = ''.join(filter(str.isdigit, isbn_candidate))

books[books["isbn13"] == int(isbn_numeric)]


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3749,9780786808380,0786808381,Baby Einstein: Babies,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=jv4NA...,"Introduce your babies to birds, cats, dogs, an...",2002.0,4.03,20.0,29.0,Baby Einstein: Babies,"9780786808380Introduce your babies to birds, c..."


In [33]:
def retrieve_semantic_recommendation(query: str, top_k: int = 10,) -> pd.DataFrame:
  recs = db_books.similarity_search(query, k=50)
  books_list=[]
  for rec in recs:
    isbn_candidate = rec.page_content.strip('"').split()[0]  # Extract first word
    isbn_numeric = ''.join(filter(str.isdigit, isbn_candidate))  # Keep only numbers

    if isbn_numeric.isdigit():  # Check if it's a valid number
        books_list.append(int(isbn_numeric))
  return books[books["isbn13"].isin(books_list)].head(top_k)

In [34]:
retrieve_semantic_recommendation("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,9780060959036Barbara Kingsolver's fifth novel ...
404,9780064402453,0064402452,Racso and the Rats of NIMH,Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=MgoNv...,"‘Racso, a brash and boastful little rodent, is...",1988.0,3.76,288.0,3231.0,Racso and the Rats of NIMH,"9780064402453‘Racso, a brash and boastful litt..."
406,9780064403870,0064403874,"R-T, Margaret, and the Rats of NIMH",Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=WTHHH...,"When Margaret and her younger brother, Artie, ...",1991.0,3.52,272.0,631.0,"R-T, Margaret, and the Rats of NIMH",9780064403870When Margaret and her younger bro...
407,9780064404419,0064404412,The Rainbow People,Laurence Yep,Juvenile Fiction,http://books.google.com/books/content?id=5AHwq...,"""Culled from 69 stories collected in a [1930s]...",1992.0,3.75,208.0,202.0,The Rainbow People,"9780064404419""Culled from 69 stories collected..."
416,9780064406925,006440692X,Winter on the Farm,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=IvlKH...,The Little House books tell the story of a lit...,1997.0,4.13,32.0,400.0,Winter on the Farm,9780064406925The Little House books tell the s...
429,9780064434980,0064434982,The Deer in the Wood,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=V7YDW...,Even the youngest child can enjoy a special ad...,1999.0,4.17,32.0,302.0,The Deer in the Wood,9780064434980Even the youngest child can enjoy...
991,9780192862099,019286209X,The Origins of Life,John Maynard Smith;Eörs Szathmáry,Science,http://books.google.com/books/content?id=nHDbB...,'I can recommend this book as a thoroughly int...,2000.0,4.11,192.0,41.0,The Origins of Life: From the Birth of Life to...,9780192862099'I can recommend this book as a t...
1639,9780374422080,0374422087,Everything on a Waffle,Polly Horvath,Juvenile Fiction,http://books.google.com/books/content?id=NimVJ...,This Newbery Honor Book tells the story of 11 ...,2004.0,3.71,150.0,9631.0,Everything on a Waffle,9780374422080This Newbery Honor Book tells the...
1642,9780374522599,0374522596,The Control of Nature,John McPhee,Nature,http://books.google.com/books/content?id=p1qKQ...,The Control of Nature is John McPhee's bestsel...,1990.0,4.24,288.0,3365.0,The Control of Nature,9780374522599The Control of Nature is John McP...
1943,9780393959055,0393959058,"Walden ; And, Resistance to Civil Government",Henry David Thoreau,Literary Collections,http://books.google.com/books/content?id=uVxnQ...,The classic chronicle of a communion with natu...,1992.0,3.95,482.0,232.0,"Walden ; And, Resistance to Civil Government: ...",9780393959055The classic chronicle of a commun...


Till now we got our basic book recomendation system .Now we want to add text classification to our model using LLMS.Its not necessary that classification will be done by llms only, we were doing this before llms.

Here we will be using Zero-Shot classification
Zero-shot classification is a powerful NLP technique where a model can classify text into categories without being explicitly trained on those categories